In [1]:
import torch 
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('mon_names.txt',encoding="utf-8").read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

2846
14
['Мустафа', 'Альфараби', 'Алфараби', 'Юсуф', 'София', 'Елиф', 'Халифа', 'Элиф']


In [3]:
len(words)

2846

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'Ё', 2: 'А', 3: 'Б', 4: 'В', 5: 'Г', 6: 'Д', 7: 'Е', 8: 'Ж', 9: 'З', 10: 'И', 11: 'К', 12: 'Л', 13: 'М', 14: 'Н', 15: 'О', 16: 'П', 17: 'Р', 18: 'С', 19: 'Т', 20: 'У', 21: 'Х', 22: 'Ц', 23: 'Ч', 24: 'Ш', 25: 'Э', 26: 'Ю', 27: 'Я', 28: 'а', 29: 'б', 30: 'в', 31: 'г', 32: 'д', 33: 'е', 34: 'ж', 35: 'з', 36: 'и', 37: 'й', 38: 'к', 39: 'л', 40: 'м', 41: 'н', 42: 'о', 43: 'п', 44: 'р', 45: 'с', 46: 'т', 47: 'у', 48: 'ф', 49: 'х', 50: 'ц', 51: 'ч', 52: 'ш', 53: 'щ', 54: 'ъ', 55: 'ы', 56: 'ь', 57: 'э', 58: 'ю', 59: 'я', 60: 'ё', 61: 'Ү', 62: 'ү', 63: 'Ө', 64: 'ө', 0: '.'}
65


In [6]:
# build the dataset
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([21722, 3]) torch.Size([21722])
torch.Size([2738, 3]) torch.Size([2738])
torch.Size([2673, 3]) torch.Size([2673])


In [7]:
Xtr.shape, Ytr.shape

(torch.Size([21722, 3]), torch.Size([21722]))

In [11]:
hidden_layer = 200
n_emb = 10

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb),               generator=g)
W1 = torch.randn((n_emb*block_size, hidden_layer), generator=g) * (5/3)/((n_emb * block_size)**0.5) #0.2
# b1 = torch.randn(hidden_layer,                     generator=g) * 0.01
W2 = torch.randn((hidden_layer, vocab_size),       generator=g) * 0.01
b2 = torch.randn(vocab_size,                       generator=g) * 0

bngain = torch.ones((1, hidden_layer))
bnbias = torch.zeros((1, hidden_layer))
bnmean_running = torch.zeros((1, hidden_layer))
bnstd_running = torch.ones((1, hidden_layer))

parameters = [C, W1, W2, b2, bngain, bnbias]
print('sum of parameters: ', sum([p.nelement() for p in parameters]))
for p in parameters:
    p.requires_grad = True

max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ))
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 # + b1
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias # BatchNorm
    
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    # lr = lrs[i]
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    #lri.append(lr)
    if i % 10000 == 0: 
        print(f"{i:7d}/{max_steps:7d}: {loss.item():4f}")
    lossi.append(loss.log10().item())
    break

sum of parameters:  20115
      0/ 200000: 4.197937


In [12]:
# Let's train a deeper network
# The classes we create here are the same API as nn.Module in PyTorch

class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)  # batch mean
            xvar = x.var(0, keepdim=True)  # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(
            xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (
                    1 -
                    self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum
                                    ) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]


class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []


n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 100  # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647)  # for reproducibility

C = torch.randn((vocab_size, n_embd), generator=g)
layers = [
    Linear(n_embd * block_size, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, vocab_size, bias=False),
    BatchNorm1d(vocab_size),
]

with torch.no_grad():
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 1.0  #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))  # number of parameters in total
for p in parameters:
    p.requires_grad = True

51280


In [13]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y
    # forward pass
    emb = C[Xb]  # embed the characters into vectors
    x = emb.view(emb.shape[0], -1)  # concatenate the vectors
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)  # loss function
    # backward pass
    for layer in layers:
        layer.out.retain_grad()  # AFTER_DEBUG: would take out retain_graph
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    with torch.no_grad():
        ud.append([((lr * p.grad).std() / p.data.std()).log10().item()
                   for p in parameters])
#     if i >= 1000:
#         break  # AFTER_DEBUG: would take out obviously to run full optimization

      0/ 200000: 4.2250
  10000/ 200000: 1.0123
  20000/ 200000: 1.0143
  30000/ 200000: 0.9209
  40000/ 200000: 1.0702
  50000/ 200000: 1.1131
  60000/ 200000: 1.1525
  70000/ 200000: 0.8692
  80000/ 200000: 0.8502
  90000/ 200000: 1.1649
 100000/ 200000: 0.9381
 110000/ 200000: 0.7214
 120000/ 200000: 0.8808
 130000/ 200000: 0.7356
 140000/ 200000: 0.7511
 150000/ 200000: 0.7755
 160000/ 200000: 1.0777
 170000/ 200000: 0.7551
 180000/ 200000: 1.0375
 190000/ 200000: 0.9658


In [14]:
# plt.figure(figsize=(20,10))
# plt.imshow(h.abs() > 0.99, cmap='gray', interpolation='nearest')

# plt.hist(h.view(-1).tolist(), 50);

# plt.hist(hpreact.view(-1).tolist(), 50);

# with torch.no_grad():
#     emb = C[Xtr]
#     embcat = emb.view(emb.shape[0], -1)
#     hpreact = embcat @ W1 + b1
#     # measure the mean/std over the entire training set
#     bnmean = hpreact.mean(0, keepdim=True)
#     bnstd = hpreact.std(0, keepdim=True)

In [15]:
@torch.no_grad()  # this decorator disables gradient tracking
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x]  # (N, block_size, n_embd)
    x = emb.view(emb.shape[0], -1)  # concat into (N, block_size * n_embd)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, y)
    print(split, loss.item())

# put layers into eval mode
for layer in layers:
    layer.training = False
split_loss('train')
split_loss('val')

train 0.8987172245979309
val 1.2836278676986694


In [16]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        # forward pass the neural net
        emb = C[torch.tensor([context])]  # (1,block_size,n_embd)
        x = emb.view(emb.shape[0], -1)  # concatenate the vectors
        for layer in layers:
            x = layer(x)
        logits = x
        probs = F.softmax(logits, dim=1)
        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the special '.' token, break
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))  # decode and print the generated word

Баянсүрэн.
Дашдончүлжнасанжав.
Нямдорж.
Айбек.
Мөнхтүвшин.
Буянзаяа.
Махфуз.
Б.
Гэрэл.
Түвшинжан.
Өнөрдарьд.
Батэрдэнэчулуунпүрэвсүрэнжаргал.
Эрдэнэ.
Уранцэцэгмасор.
Болорчимэг.
Тэргүй.
Алтантуяа.
Өлзийтолкынжав.
Цэенов.
Долгорсүрэн.
